In [1]:
import pandas as pd
from datetime import datetime
import json

import warnings
warnings.filterwarnings('ignore')

fase = '04_crear_train_test'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

path_lag = gen_config['path_lag']

path_train = gen_config['path_train']
path_test = gen_config['path_test']
path_futuro = gen_config['path_futuro']

periodo_train_start = gen_config['var_periodo_train_start']
periodo_train_end = gen_config['var_periodo_train_end']
periodo_test = gen_config['var_periodo_test']
periodo_futuro = gen_config['var_periodo_futuro']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
----------------------------------------04_crear_train_test-----------------------------------------


In [4]:
df = pd.read_parquet(f"{folder}/{path_lag}")
print(f"Tamaño datos con features: {df.shape}")

Tamaño datos con features: (2293481, 196)


In [5]:
df.drop(columns=['tn','values','total_tn','average_tn','median_tn','std_dev_tn','iqr_tn','max_tn','min_tn'], inplace=True)

In [6]:
df['primer_periodo'] = pd.to_datetime(df['primer_periodo']).dt.strftime('%Y%m').astype(int)
df['ultimo_periodo'] = pd.to_datetime(df['ultimo_periodo']).dt.strftime('%Y%m').astype(int)
df['periodo_dt'] = pd.to_datetime(df['periodo_dt']).dt.strftime('%Y%m').astype(int)
df['tn_futuro'] = -df.groupby(['product_id','customer_id'])['tn_norm'].diff(-2)

In [7]:
df_train = df[(df['periodo']>=periodo_train_start) & (df['periodo']<=periodo_train_end)]
df_test  = df[df['periodo']==periodo_test]
df_futuro  = df[df['periodo']==periodo_futuro]

In [8]:
print(f"{'Shape Train':.<25}: {df_train.shape} para los periodos desde {periodo_train_start} hasta {periodo_train_end}")
print(f"{'Shape Test':.<25}: {df_test.shape} para el periodo {periodo_test}")
print(f"{'Shape Futuro':.<25}: {df_futuro.shape} para el periodo {periodo_futuro}")

Shape Train..............: (2173865, 188) para los periodos desde 201701 hasta 201910
Shape Test...............: (68823, 188) para el periodo 201910
Shape Futuro.............: (53008, 188) para el periodo 201912


In [9]:
# df_train.to_parquet('sell_in_train.parquet', index=False)
# df_test.to_parquet('sell_in_test.parquet', index=False)
# df_futuro.to_parquet('sell_in_futuro.parquet', index=False)

df_train.to_parquet(f'{folder}/{path_train}', index=False)
df_test.to_parquet(f'{folder}/{path_test}', index=False)
df_futuro.to_parquet(f'{folder}/{path_futuro}', index=False)

In [10]:
#estado_control = f"04_crear_train_set Terminado - {nombrefile} - {datetime.now()}"

In [11]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

----------------------------------------04_crear_train_test-----------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



